In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
from absl import app
from absl import flags
import numpy as np
import tensorflow as tf


from tensorflow.contrib.tpu.python.tpu import async_checkpoint
from tensorflow.contrib.training.python.training import evaluation
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.python.estimator import estimator
from tensorflow.python.keras import backend as K


In [2]:
import sys
sys.path.append('../nas-search')
import search_main, imagenet_input
from search_main import nas_model_fn


FLAGS = flags.FLAGS
FLAGS(["data_format, input_image_size, num_label_classes, num_eval_images, eval_batch_size, num_train_images, train_batch_size, use_tpu"])
flags.FLAGS.data_format="channels_last"
flags.FLAGS.input_image_size = 64
flags.FLAGS.num_label_classes =200
#flags.FLAGS.num_label_classes =10
flags.FLAGS.num_eval_images=9832
flags.FLAGS.eval_batch_size=1024
flags.FLAGS.num_train_images=100000
flags.FLAGS.train_batch_size=1024
flags.FLAGS.use_tpu=False

# Hyperparameters

In [3]:
model_dir = "/Users/roxanefischer/Desktop/models_tiny_imagenet_v2"
data_dir = "/Users/roxanefischer/Documents/cours/3A/Stage_ML/single-path-nas/other/tiny_imagenet/all"
ckpt ="/Users/roxanefischer/Desktop/models_tiny_imagenet_v2/model.ckpt-10000"
#ckpt="/Users/roxanefischer/Desktop/cifar/models/model.ckpt-6"

save_checkpoints_steps=10000
log_step_count_steps=64
iterations_per_loop=10000
num_parallel_calls=64

num_train_images=100000
train_batch_size=1024

num_eval_images=9832
eval_batch_size=1024

predict_batch_size=3

use_tpu=False
export_to_tpu=False
transpose_input = True
use_cache=True

#tiny_image_net
input_image_size=64

# Define NAS Model

In [4]:
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver('',zone=None, project=None)
  
  config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      model_dir=model_dir,
      save_checkpoints_steps=save_checkpoints_steps,
      log_step_count_steps=log_step_count_steps,
      session_config=tf.ConfigProto(
          graph_options=tf.GraphOptions(
              rewrite_options=rewriter_config_pb2.RewriterConfig(
                  disable_meta_optimizer=True))),
      tpu_config=tf.contrib.tpu.TPUConfig(
          iterations_per_loop=iterations_per_loop,
          per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig
          .PER_HOST_V2))  # pylint: disable=line-too-long
  # Initializes model parameters.
  params = dict(steps_per_epoch=num_train_images / train_batch_size)
  nas_est = tf.contrib.tpu.TPUEstimator(
      use_tpu=use_tpu,
      model_fn=nas_model_fn,
      config=config,
      train_batch_size=train_batch_size,
      eval_batch_size=eval_batch_size,
      predict_batch_size=predict_batch_size,
      export_to_tpu=export_to_tpu,
      params=params)

INFO:tensorflow:Using config: {'_model_dir': '/Users/roxanefischer/Desktop/models_tiny_imagenet_v2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 10000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': None, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': b'', '_evaluation_master': b'', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=10000, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_cluster': <tensorflow.contrib.

# Load datasets

In [5]:
imagenet_train, imagenet_eval = [
    imagenet_input.ImageNetInput(
        is_training=is_training,
        data_dir=data_dir,
        transpose_input=transpose_input,
        cache=use_cache and is_training,
        image_size=input_image_size,
        num_parallel_calls=num_parallel_calls,
        use_bfloat16=False) for is_training in [True, False]
]

# Evaluation on train and test sets

In [6]:
eval_steps = num_eval_images // eval_batch_size
def eval_results(ckpt ="/Users/roxanefischer/Desktop/models_tiny_imagenet_v2/model.ckpt-10000", eval_steps = eval_steps):
    eval_results = nas_est.evaluate(
        input_fn=imagenet_eval.input_fn,
        steps=eval_steps,
        checkpoint_path=ckpt)
    return eval_results

train_steps = num_train_images // train_batch_size
def train_results(ckpt ="/Users/roxanefischer/Desktop/models_tiny_imagenet_v2/model.ckpt-10000", train_steps = train_steps):
    train_results = nas_est.evaluate(
        input_fn=imagenet_train.input_fn,
        steps=train_steps,
        checkpoint_path=ckpt)
    return train_results

# Results

## NAS Search

## Checkpoint 10000 on TinyImagenet

### Eval results

{ALL global_step = 10000, loss = 407.13586, **top_1_accuracy = 0.006184896, top_5_accuracy = 0.024631076**}

### Train results 

{ALL global_step = 10000, loss = 403.2064, **top_1_accuracy = 0.35014296, top_5_accuracy = 0.60899246**}


## Checkpoint 785 on TinyImagenet

### Eval results

{1  : global_step = 785, loss = 498.46667, **top_1_accuracy = 0.0048828125, top_5_accuracy = 0.0234375**}

{All :global_step = 785, loss = 498.53845, **top_1_accuracy = 0.0046657985, top_5_accuracy = 0.025282118**}

### Train results 

{1 : global_step = 785, loss = 496.75244, **top_1_accuracy = 0.022460938, top_5_accuracy = 0.064453125**}

{All : global_step = 785, loss = 496.75757, top_1_accuracy = 0.017386839, top_5_accuracy = 0.055271424}



### 1 eval_step on eval_dataset model ckpt = 785
{'loss': 498.56723,
 'top_1_accuracy': 0.005859375,
 'top_5_accuracy': 0.025390625,
 'global_step': 785}

 {'loss': 498.4192,
 'top_1_accuracy': 0.0068359375,
 'top_5_accuracy': 0.025390625,
 'global_step': 785}

{'loss': 498.5098,
 'top_1_accuracy': 0.0029296875,
 'top_5_accuracy': 0.02734375,
 'global_step': 785}

{'loss': 498.39905,
 'top_1_accuracy': 0.005859375,
 'top_5_accuracy': 0.024414062,
 'global_step': 785}

 

In [7]:

predictions = nas_est.predict(input_fn=imagenet_eval.input_fn,checkpoint_path=ckpt,yield_single_examples=True)
c=0
for i in predictions:
    print(i)
    c=c+1
    if (c>3):
        break

4, 4, 1, 480)
INFO:tensorflow:DWConv: single-path-search/single_path_super_net/mnas_blocks_10/Relu_1:0 shape: (64, 4, 1, 480)
INFO:tensorflow:Project: single-path-search/single_path_super_net/mnas_blocks_10/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Block input: single-path-search/single_path_super_net/mnas_blocks_10/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Expand: single-path-search/single_path_super_net/mnas_blocks_11/Relu:0 shape: (64, 4, 1, 480)
INFO:tensorflow:DWConv: single-path-search/single_path_super_net/mnas_blocks_11/Relu_1:0 shape: (64, 4, 1, 480)
INFO:tensorflow:Project: single-path-search/single_path_super_net/mnas_blocks_11/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Block input: single-path-search/single_path_super_net/mnas_blocks_11/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Expand: single-path-search/single_path_super_net/mnas_blocks_12/Relu:0 shape: (64, 4, 1, 480)
INFO:tensorflow:DWConv: single-path-search/single_path_super_net/mnas_blocks_12/Relu_1:0 shape: (6

In [8]:

predictions = nas_est.predict(input_fn=imagenet_train.input_fn,checkpoint_path=ckpt,yield_single_examples=True)
c=0
for i in predictions:
    print(i)
    c=c+1
    if (c>3):
        break

4, 4, 1, 480)
INFO:tensorflow:DWConv: single-path-search/single_path_super_net/mnas_blocks_10/Relu_1:0 shape: (64, 4, 1, 480)
INFO:tensorflow:Project: single-path-search/single_path_super_net/mnas_blocks_10/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Block input: single-path-search/single_path_super_net/mnas_blocks_10/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Expand: single-path-search/single_path_super_net/mnas_blocks_11/Relu:0 shape: (64, 4, 1, 480)
INFO:tensorflow:DWConv: single-path-search/single_path_super_net/mnas_blocks_11/Relu_1:0 shape: (64, 4, 1, 480)
INFO:tensorflow:Project: single-path-search/single_path_super_net/mnas_blocks_11/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Block input: single-path-search/single_path_super_net/mnas_blocks_11/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Expand: single-path-search/single_path_super_net/mnas_blocks_12/Relu:0 shape: (64, 4, 1, 480)
INFO:tensorflow:DWConv: single-path-search/single_path_super_net/mnas_blocks_12/Relu_1:0 shape: (6

In [9]:
probabilities = np.array([2.20111891e-04, 6.21380401e-04, 1.65894107e-05, 1.54913396e-05,
       1.37975067e-05, 1.45058502e-05, 1.55343628e-03, 6.32928204e-05,
       2.05146105e-04, 2.66212097e-04, 7.59928254e-03, 7.34552086e-05,
       2.11017414e-05, 2.89958931e-04, 5.23802941e-04, 4.48159117e-04,
       1.00495096e-03, 3.20559461e-03, 1.33768743e-04, 1.05031140e-04,
       3.61345301e-05, 1.25022843e-05, 1.15304982e-04, 4.07184889e-05,
       3.92045877e-05, 3.15749785e-05, 3.05831818e-05, 1.12743282e-05,
       1.39526674e-05, 2.31932390e-05, 9.37903405e-06, 5.29259210e-04,
       7.85046068e-05, 8.17779801e-05, 1.17297768e-05, 3.22870947e-05,
       7.53025641e-04, 5.84401823e-05, 8.69072508e-04, 8.80408508e-04,
       1.63389996e-04, 1.47625559e-03, 1.16826093e-03, 3.28927243e-04,
       1.09937873e-05, 1.14944531e-04, 2.08328408e-03, 1.71310545e-04,
       4.60983545e-04, 3.61297061e-05, 3.54095791e-05, 4.72538741e-05,
       9.62779086e-05, 8.43219241e-05, 3.36341094e-04, 9.97825264e-05,
       4.32493071e-05, 2.47625994e-05, 1.32011090e-04, 4.76012468e-01,
       4.88431797e-05, 5.18020999e-04, 8.18631519e-03, 7.26981525e-05,
       4.19734890e-04, 4.78269393e-03, 1.48435320e-05, 1.23759301e-03,
       3.55904667e-05, 3.97635631e-05, 5.15784959e-05, 1.29270979e-04,
       2.14259824e-04, 6.16202084e-03, 7.31303880e-05, 1.36256076e-05,
       5.17101726e-04, 1.56632115e-04, 1.50104426e-03, 5.92659647e-03,
       1.04469468e-03, 5.09349702e-05, 3.03668960e-04, 4.82242648e-03,
       8.84173824e-06, 5.75339014e-04, 1.30805769e-04, 6.20622304e-05,
       5.15075109e-04, 4.90186321e-05, 1.38232503e-02, 6.52345334e-05,
       7.20422831e-05, 3.09206657e-02, 1.49120126e-04, 2.98288325e-03,
       1.88197533e-04, 8.58127460e-05, 1.01259866e-04, 5.17187254e-05,
       1.95636621e-05, 2.05598539e-04, 6.12803386e-04, 1.83405413e-04,
       1.24263810e-04, 5.36340987e-04, 2.20421090e-04, 2.58578832e-04,
       2.50177603e-04, 4.31851331e-05, 9.35988151e-04, 4.76613559e-06,
       2.07811799e-02, 2.67670699e-03, 4.15132745e-05, 5.62884306e-05,
       4.19571437e-03, 1.56111215e-04, 4.10694852e-02, 2.39748610e-04,
       1.38143136e-04, 1.69828898e-04, 2.53555827e-05, 2.46135169e-05,
       1.69305038e-03, 3.94349190e-04, 2.80999229e-05, 1.73843495e-04,
       3.38300713e-03, 6.04907509e-05, 1.97184202e-03, 4.67496284e-04,
       1.20210545e-02, 7.21530523e-05, 2.47405772e-03, 3.67168360e-03,
       3.88118169e-05, 1.51630302e-04, 3.51255434e-03, 2.38899356e-05,
       2.40968214e-03, 3.23847309e-02, 1.52760604e-05, 3.10511590e-04,
       9.06876012e-05, 6.33713935e-05, 2.54389993e-03, 1.65094229e-04,
       1.16669937e-04, 2.01372579e-01, 5.56001149e-04, 2.60033557e-04,
       3.26044741e-04, 1.16479394e-04, 1.72887303e-04, 4.80192330e-06,
       2.42511542e-05, 2.17002846e-04, 1.49966552e-04, 3.33079486e-04,
       3.50085575e-05, 1.18037337e-04, 6.74785770e-05, 2.43224483e-03,
       9.45210832e-06, 8.50612287e-06, 5.71980236e-05, 5.40055917e-04,
       4.58553986e-04, 5.26543893e-02, 3.47189671e-05, 1.08353612e-04,
       3.13968887e-03, 1.05858999e-05, 9.74112190e-05, 2.43306597e-04,
       4.83278738e-04, 1.86935024e-04, 9.32660987e-05, 6.05468813e-04,
       1.64106395e-03, 9.12834483e-04, 4.13171765e-05, 5.79637235e-05,
       3.34642937e-05, 4.43052595e-05, 1.69696956e-04, 1.18811122e-05,
       5.92623983e-05, 3.31994583e-04, 3.02266912e-04, 1.09902707e-04,
       3.28223541e-05, 5.15918728e-05, 4.01323050e-05, 8.31878424e-06,
       6.17814716e-04, 1.71617387e-04, 1.39998185e-04, 1.27410303e-05])
prob_2=np.array([2.17094319e-04, 7.53633678e-04, 3.56615383e-05, 3.22067317e-05,
       4.56569760e-05, 3.34203723e-05, 2.55392911e-03, 1.16655407e-04,
       3.52568604e-04, 5.69190830e-04, 1.01034474e-02, 1.53011293e-04,
       4.12718800e-05, 4.30395419e-04, 4.50078544e-04, 8.89140647e-04,
       1.10128790e-03, 2.16715247e-03, 2.44421011e-04, 1.43244673e-04,
       7.11492612e-05, 3.35100849e-05, 1.95820088e-04, 8.90097217e-05,
       7.01790559e-05, 5.13292107e-05, 5.73592661e-05, 1.83613120e-05,
       3.08531889e-05, 4.70443301e-05, 3.78181612e-05, 9.91325127e-04,
       2.34816078e-04, 1.90448511e-04, 1.84656383e-05, 7.01209210e-05,
       1.01028115e-03, 1.25924285e-04, 8.47837538e-04, 9.60929669e-04,
       4.62999393e-04, 2.44170101e-03, 1.87580928e-03, 5.34155406e-04,
       1.73134431e-05, 1.61296412e-04, 2.57757073e-03, 3.77597462e-04,
       8.62023444e-04, 6.48273781e-05, 5.89156043e-05, 8.69935975e-05,
       1.52071676e-04, 1.47567669e-04, 6.01813139e-04, 2.30391743e-04,
       9.45288630e-05, 2.47010357e-05, 2.02411000e-04, 2.94179261e-01,
       1.13437185e-04, 7.81905605e-04, 1.04738381e-02, 1.08499087e-04,
       8.79520492e-04, 8.21342971e-03, 1.85381050e-05, 1.23766123e-03,
       7.36037837e-05, 6.16866964e-05, 9.80294426e-05, 3.20722698e-04,
       2.99208681e-04, 7.76415505e-03, 8.89890143e-05, 2.40265636e-05,
       4.99483140e-04, 2.57696258e-04, 3.82595765e-03, 7.74894981e-03,
       1.01684337e-03, 8.14920859e-05, 3.88215703e-04, 5.73055353e-03,
       1.33203821e-05, 1.20071089e-03, 3.81959573e-04, 1.08738059e-04,
       7.23359641e-04, 5.88240910e-05, 1.60154421e-02, 9.14673874e-05,
       2.01375224e-04, 1.55233871e-02, 2.55605730e-04, 3.36023956e-03,
       3.38587881e-04, 2.05212404e-04, 2.04010968e-04, 8.68746210e-05,
       3.42028652e-05, 2.58891552e-04, 1.06353755e-03, 3.75960401e-04,
       1.42032921e-04, 1.33452157e-03, 3.70844151e-04, 3.39978229e-04,
       5.82324108e-04, 9.72945927e-05, 2.02415232e-03, 1.51840895e-05,
       1.77865867e-02, 2.84027797e-03, 6.73113900e-05, 9.60410107e-05,
       5.11983363e-03, 2.50155019e-04, 2.93656215e-02, 3.80243728e-04,
       3.10737843e-04, 2.73376179e-04, 4.10371758e-05, 9.11459938e-05,
       3.52909393e-03, 6.22601074e-04, 4.80632334e-05, 3.45194974e-04,
       8.41495208e-03, 1.30727800e-04, 3.19762505e-03, 6.83449558e-04,
       1.54982582e-02, 9.56682270e-05, 3.83665971e-03, 4.51403810e-03,
       8.70644799e-05, 2.15127104e-04, 5.90379676e-03, 2.34844083e-05,
       4.80121095e-03, 1.11438550e-01, 2.83483841e-05, 5.84273308e-04,
       1.83816970e-04, 1.16195399e-04, 4.10926715e-03, 2.24408228e-04,
       2.01325005e-04, 2.79742658e-01, 6.34081778e-04, 7.38886534e-04,
       6.24762557e-04, 2.33590879e-04, 3.18899722e-04, 1.40924913e-05,
       8.20881687e-05, 4.10491339e-04, 1.73233537e-04, 5.07705321e-04,
       5.55921761e-05, 2.04310418e-04, 1.31760477e-04, 4.58548684e-03,
       1.32810583e-05, 1.42656017e-05, 6.50372385e-05, 1.32960081e-03,
       5.82780805e-04, 4.55020815e-02, 5.96596183e-05, 1.34263144e-04,
       3.32529237e-03, 2.05246834e-05, 1.38056916e-04, 5.24510222e-04,
       6.25545625e-04, 2.11221035e-04, 1.21835677e-04, 8.60757951e-04,
       2.12302082e-03, 1.24898960e-03, 6.34999451e-05, 7.40313044e-05,
       4.90925449e-05, 6.20623978e-05, 2.04584750e-04, 2.10591570e-05,
       1.19446864e-04, 2.62360438e-04, 3.60841601e-04, 1.15820127e-04,
       6.77105709e-05, 8.58803542e-05, 9.96663875e-05, 1.48301860e-05,
       7.56966940e-04, 3.21493339e-04, 2.22442410e-04, 2.96593407e-05])


In [10]:
from operator import itemgetter
from heapq import nlargest
result = nlargest(50, enumerate(probabilities), itemgetter(1))

In [11]:
result

[(59, 0.476012468),
 (149, 0.201372579),
 (169, 0.0526543893),
 (118, 0.0410694852),
 (141, 0.0323847309),
 (93, 0.0309206657),
 (112, 0.0207811799),
 (90, 0.0138232503),
 (132, 0.0120210545),
 (62, 0.00818631519),
 (10, 0.00759928254),
 (73, 0.00616202084),
 (79, 0.00592659647),
 (83, 0.00482242648),
 (65, 0.00478269393),
 (116, 0.00419571437),
 (135, 0.0036716836),
 (138, 0.00351255434),
 (128, 0.00338300713),
 (17, 0.00320559461),
 (172, 0.00313968887),
 (95, 0.00298288325),
 (113, 0.00267670699),
 (146, 0.00254389993),
 (134, 0.00247405772),
 (163, 0.00243224483),
 (140, 0.00240968214),
 (46, 0.00208328408),
 (130, 0.00197184202),
 (124, 0.00169305038),
 (180, 0.00164106395),
 (6, 0.00155343628),
 (78, 0.00150104426),
 (41, 0.00147625559),
 (67, 0.00123759301),
 (42, 0.00116826093),
 (80, 0.00104469468),
 (16, 0.00100495096),
 (110, 0.000935988151),
 (181, 0.000912834483),
 (39, 0.000880408508),
 (38, 0.000869072508),
 (36, 0.000753025641),
 (1, 0.000621380401),
 (196, 0.0006178147

In [12]:
nlargest(50, enumerate(prob_2), itemgetter(1))

[(59, 0.294179261),
 (149, 0.279742658),
 (141, 0.11143855),
 (169, 0.0455020815),
 (118, 0.0293656215),
 (112, 0.0177865867),
 (90, 0.0160154421),
 (93, 0.0155233871),
 (132, 0.0154982582),
 (62, 0.0104738381),
 (10, 0.0101034474),
 (128, 0.00841495208),
 (65, 0.00821342971),
 (73, 0.00776415505),
 (79, 0.00774894981),
 (138, 0.00590379676),
 (83, 0.00573055353),
 (116, 0.00511983363),
 (140, 0.00480121095),
 (163, 0.00458548684),
 (135, 0.0045140381),
 (146, 0.00410926715),
 (134, 0.00383665971),
 (78, 0.00382595765),
 (124, 0.00352909393),
 (95, 0.00336023956),
 (172, 0.00332529237),
 (130, 0.00319762505),
 (113, 0.00284027797),
 (46, 0.00257757073),
 (6, 0.00255392911),
 (41, 0.00244170101),
 (17, 0.00216715247),
 (180, 0.00212302082),
 (110, 0.00202415232),
 (42, 0.00187580928),
 (105, 0.00133452157),
 (167, 0.00132960081),
 (181, 0.0012489896),
 (67, 0.00123766123),
 (85, 0.00120071089),
 (16, 0.0011012879),
 (102, 0.00106353755),
 (80, 0.00101684337),
 (36, 0.00101028115),
 (31,

In [13]:
train_results_v2_0 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v2/model.ckpt-0", train_steps = 10)
train_results_v2_200 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v2/model.ckpt-200", train_steps = 10)
train_results_v2_400 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v2/model.ckpt-400", train_steps = 10)

train_results_v3_0 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v3/model.ckpt-0", train_steps = 10)
train_results_v3_200 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v3/model.ckpt-200", train_steps = 10)
train_results_v3_400 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v3/model.ckpt-400", train_steps = 10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:Dropout rate warmup steps: 491
INFO:tensorflow:Runtime model parsed
INFO:tensorflow:Built stem layers with output shape: (1024, 32, 32, 32)
INFO:tensorflow:Block input: single-path-search/single_path_super_net/mnas_stem/Relu:0 shape: (1024, 32, 32, 32)
INFO:tensorflow:Expand: single-path-search/single_path_super_net/mnas_stem/Relu:0 shape: (1024, 32, 32, 32)
INFO:tensorflow:DWConv: single-path-search/single_path_super_net/mnas_blocks_0/Relu:0 shape: (1024, 32, 32, 32)
INFO:tensorflow:Project: single-path-search/single_path_super_net/mnas_blocks_0/batch_normalization_1/FusedBatchNorm:0 shape: (1024, 32, 32, 16)
INFO:tensorflow:Block input: single-path-search/single_path_super_net/mnas_blocks_0/batch_normalization_1/FusedBatchNorm:0 shape: (1024, 32, 32, 16)
INFO:tensorflow:Expand: single-path-search/single_path_super_net/mnas_blocks_1/Relu:0 shape: (1024, 32, 32, 96)
INFO:tensorflow:DWConv: single-path

KeyboardInterrupt: 